In [1]:
import cv2
import numpy as np
#import bimpy
import time
from PIL import ImageFont, ImageDraw, Image
#from find_circles import find_circles
from threading import Thread
from sklearn.metrics import r2_score
from Stack import *

import math
current_milli_time = lambda: int(round(time.time() * 1000))

In [2]:
def distf(a,b):
    return math.sqrt(math.pow(a[0]-b[0],2)+math.pow(a[1]-b[1],2))

def move_to_storage():
    pass

def clean_place(x,y):
    pass

def test(m):
    print(m)
    
def clean_robot_plate(x,y):
    pass

def robot():
    global robot_work
    global turning_cutlets
    
    while True:
        if not turning_cutlets.is_empty():
            robot_work = True
            k = turning_cutlets.pop()
            k['turning'] = True
            x,y,r = k['pos']
            turn_cutlet(k)
            k['turning'] = False
            clean_place(x,y)
            robot_work = False
            print("kuku")
        time.sleep(0.1)
    
class RoboThread(Thread):    
    def __init__(self):
        Thread.__init__(self)
        self.stoped = True

    def run(self):
        global robot_work
        global turning_cutlets

        while self.stoped:
            if not turning_cutlets.is_empty():
                robot_work = True
                k = turning_cutlets.pop()
                k['turning'] = True
                x,y,r = k['pos']
                turn_cutlet(k)
                k['turning'] = False
                clean_place(x,y)
                robot_work = False
                print("kuku")
            time.sleep(0.1)
        #print("Stoped")
    def stop(self):
        self.stoped = False

def add_cutlet(x,y,r):
    global table
#     global start_time
    global order_number
    
    kotleta = {
    "pos": (x,y,r),
    "order": order_number,
    "fry":[
        [0,current_milli_time()],
    ],
    "status": 0.0,
    "turning":True,
    "r2score":0
    
   
    }
        
    table = np.append(table, kotleta)
    order_number += 1
    
    
    
def turn_cutlet(katleta):
#     global start_time
    fry = katleta['fry']
    x,y,r = katleta['pos']
    if len(fry)<4:
        fry[-1][1]=0.0
        if fry[-1][0] == 1:
            fry.append([0, current_milli_time()])
        else:
            fry.append([1,current_milli_time()])
        
            
          
        robot_turn(x,y)
        
        circles = find_circles(frame) 
        for x,y,r in circles:
            found = False
            for k in table:
                
                x_t,y_t,r_t = k['pos']
                
                if distf((x,y),(x_t+r*2,y_t))<150:
                    k['pos'] = (x,y,r)
                    found = True
                    break
            if not found:
                add_cutlet(x,y,r)                
    
    else:
        move_to_storage()

In [3]:
def auto_canny(frame,sigma=0.33):
    v = np.median(frame)
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    return cv2.Canny(frame,lower,upper)

def find_circles(img,max_mean=0.3,circle_mean=0.05):
    """
        max_mean - Максимальное отличие радиусов. в процентах дефол 30%
        circle_mean - Максимальное отличие кол-ва вершин. строится градиент.
    """
    
    canny = auto_canny(img)
    canny2 = cv2.dilate(canny, np.ones((3, 3), np.uint8))
    contours, _ = cv2.findContours(canny2, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    all_radius = []
    for i,cnt in enumerate(contours):
        cnt = cv2.convexHull(cnt)
        radius = cv2.arcLength(cnt,True) / (2*np.pi)
        all_radius.append(radius)
    
    radius_average = np.average(all_radius,weights=all_radius)
    
    circles = []
    loss = lambda x: x if (x < 1) else x-1
    
    
    for i,cnt in enumerate(contours):
        cnt = cv2.convexHull(cnt)
        radius = cv2.arcLength(cnt,True) / (2*np.pi)
        if radius >= radius_average and len(cnt) > 5:#and len(cnt) > 5
            ellipse = cv2.fitEllipse(cnt)
            if loss(np.average(ellipse[1],weights=ellipse[1])/np.mean(ellipse[1])) > circle_mean:
                continue
            
            (x,y),radius2 = cv2.minEnclosingCircle(cnt)
            circles.append(np.int0([x,y,np.mean([radius,radius2]),len(cnt)]))
    
    circles = np.array(circles)
    if len(circles) > 0:
        cntlen_average = np.average(circles[:,3],weights=circles[:,3])
        circles = circles[circles[:,3]/cntlen_average > 0.5]
    
        return circles[:,:3]
    else:
        return circles

In [ ]:
cap = cv2.VideoCapture("http://192.168.1.22:8080/video")#"http://192.168.1.65:8080/video""http://192.168.1.22:8080/video"

fourcc = cv2.VideoWriter_fourcc(*'MP4V')    
ret, frame = cap.read() 
v_out = None

turning_cutlets = Stack()
order_number = 1000
start_time = current_milli_time()
fry_time = 20.0*1000
all_time = fry_time*4
robot_work = False

thread = RoboThread()
thread.start()


#инициализация bimpy
# ctx = bimpy.Context()
# ctx.init(1270, 720, "Cutlet")
#шрифт для PIL
font = ImageFont.truetype("arial.ttf", 32)

#создаем стол
table = np.array([])

try:
    #первый поиск
    assert ret == True, "НЕТ КАДРА"

    circles = find_circles(frame) 
    # print(ret,circles)
    # cap.release()

    for x, y, r in circles:
        if r>150:
            add_cutlet(x,y,r)
    
    print(len(circles))
    while True:
        #получаем кадр, преобразовываем в PIL 
        key = cv2.waitKey(1)
        ret, frame = cap.read()
        h,w = frame.shape[:2]
        pil_im = Image.fromarray(frame)
        draw = ImageDraw.Draw(pil_im)

        #создаем фрейм bimby 
        #ctx.new_frame()
        #if bimpy.begin("Cutlet", flags=(bimpy.WindowFlags.AlwaysAutoResize | bimpy.WindowFlags.NoTitleBar)): 
            #выводим готовящиеся котлеты на столе

        for kotleta in table:
            
            if not kotleta['turning']:#kotleta['status']<1 and not 

                status = kotleta['status']
                x,y,r = kotleta['pos']
                fry = kotleta['fry']
                order = "Заказ №" + str(kotleta['order'])
                #score = kotleta['r2score']
                
                indent = 0
                turn_text_size_h=0
                draw.rectangle((x-r+5, y + r - len(fry)*30, x+r-5, y+r+10), outline="white", fill="white")
                for i,f in enumerate(fry):
                    turn_text_size_w=0
                    time_c = current_milli_time()-f[1]
                    if i == len(fry)-1:
                        
                        time_text = " {}:{} ".format(int(time_c/1000/60), int(time_c/1000%60) if time_c/1000%60>10 else '0'+ str(int(time_c/1000%60)))
                    else:
                        time_text = " 0:00 "
                    #time_text = " {}:{} ".format(int(time_c/1000/60), int(time_c/1000%60) if time_c/1000%60>=9 else '0'+ str(int(time_c/1000%60)))
                    turn_text_size_w += draw.textsize(time_text, font=font)[0] + draw.textsize(order, font=font)[0]
                    turn_text_size_h += draw.textsize(order, font=font)[1]

                    side = "Сторона А " if f[0] == 0 else "Сторона Б "

                    draw.text((x-draw.textsize(side, font=font)[0]/2-30,y+r-30-indent), side + time_text,(0,0,0), font=font)
                    indent += 30


                #отрисовываем
                draw.rectangle((x-r-10, y-r-10, x+r+10, y+r+10), outline="green", width=5) #котлета   
                draw.rectangle((x-r-10, y+r+15, x+r+10, y+r+55), outline="green", width=5) #статус (пустой)
                draw.rectangle((x-r-10, y+r+15, x-r+(status*r*2), y+r+55), outline="green", fill="green") #статус 

                #draw.text((x-15, y+r+20), "{}:{} ".format(int(timer/60), int(t) if t>9 else '0'+ str(int(t)) ), font=font)   
                draw.rectangle((x-r-10, y-r-60, x+r+10, y-r-15), outline="green", width=5) #область заказа
                #draw.rectangle((x+r+15, y-r-10, x+r+turn_text_size_w+10, y-r+turn_text_size_h+10), outline="green", width=5) #переворачивания 
                draw.text((x-r,y-r-60), order, font=font) #заказ
                
                kotleta['status']+= 1/time_c
                    
                
                print(kotleta['status'])
                fry_size = len(fry)

                if (fry_size==4 and current_milli_time()-fry[-1][1]>=fry_time):
                    top = draw.textsize(time_text, font=font)[1]*5
                    draw.text((x+r+30, y-r-10+top), "Котлета готова!" , font=font)       
                

                if current_milli_time()-fry[-1][1]>=fry_time:
            
                    turning_cutlets.push(kotleta)

                if not robot_work:

                    circles = find_circles(frame) 

                    for x,y,r in circles:
                        if 130<r<160:
                            found = False
                            for k in table:
                                x_t,y_t,r_t = k['pos']

                                if distf((x,y),(x_t,y_t))<150:
                                    score = r2_score((x,y,r),(x_t,y_t,r_t))
                                    k['r2score'] = f" {score:.3f}" 

                                    if score > 0.98:
                                        found = True
                                        break

                                    k['pos'] = (x,y,r)
                                    found = True
                                    break
                            if not found:
                                add_cutlet(x,y,r) 
            if v_out is None:
                v_out = cv2.VideoWriter('interface_thread {}.mp4'.format(time.time()),fourcc, 5.0, (frame.shape[1],frame.shape[0]),3)

            
            v_out.write(np.array(pil_im))
            #изменяем размер и палитру
            resized = cv2.resize(np.array(pil_im),(int(w/2),int(h/2)))
            #resized = cv2.resize(np.array(pil_im)[:, :, ::-1],(int(w/2),int(h/2)))
            #im = bimpy.Image(resized)
            #выводим на экран
            
            
            cv2.imshow("interface",resized)

    #     bimpy.end()
    #     ctx.render()
        del draw

        #time.sleep(1)

        if key == ord('q') or ret == False:
            break 
except Exception as e:
    print(e)

if v_out is not None: v_out.release()
#if isinstance(v_out, type(None)) == False: v_out.release()
cap.release()
cv2.destroyAllWindows()
thread.stop()

2
0.0033112582781456954
0.00044072278536800354
0.0037214305505000843
0.000826525254503806
0.0040362920114572634
0.00112909711532075
0.004290550846951797
0.0013752206693448702
0.004523000870196799
0.001601465013236273
0.004741532338728268
0.001813284542996917
0.004946031316233381
0.002012845508871992
0.005139867330964918
0.002202096076623695
0.005323724657679388
0.0023817905959408556
0.005498794685690592
0.0025532877662375456
0.0056640839418889396
0.0027154411633512152
0.005791488706827148
0.002840990441442866
0.005914082797791964
0.0029620558167455296
0.006020026224002367
0.0030666911625653473
0.006123225398408972
0.0031685967979469837
0.006223375623746979
0.0032674110271959955
0.006316772466933679
0.0033598068509047637
0.0064019368341624305
0.0034441025602531578
0.006484972622587242
0.003526238083867121
0.006565429618323021
0.003605836905804556
0.0066440459705242785
0.003683730658725572
0.006720721327064687
0.0037596665688174546
0.006795717577252177
0.003833938701913117
0.006868492484

In [ ]:
v_out.release()